In [1]:
import tensorflow as tf
import numpy as np

#CIFAR-10 분류를 위해 keras를 이용해 dataset을 import 한다.
#from tensorflow.python.keras._impl.keras.datasets.cifar10 import load_data
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
def CIFAR(x):

  load_image = x

  #random_normal < truncated_normal
  W1 = tf.Variable(tf.truncated_normal([5, 5, 3, 64], stddev=5e-2))
  b1 = tf.Variable(tf.constant(0.1, shape=[64]))
  L1 = tf.nn.relu(tf.nn.conv2d(load_image, W1, strides=[1, 1, 1, 1], padding='SAME') + b1)
  L1 = tf.nn.max_pool(L1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    
  W2 = tf.Variable(tf.truncated_normal([5, 5, 64, 64], stddev=5e-2))
  b2 = tf.Variable(tf.constant(0.1, shape=[64]))
  L2 = tf.nn.relu(tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME') + b2)
  L2 = tf.nn.max_pool(L2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

  W3 = tf.Variable(tf.truncated_normal([5, 5, 64, 128], stddev=5e-2))
  b3 = tf.Variable(tf.constant(0.1, shape=[128]))
  L3 = tf.nn.relu(tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME') + b3)
    
  W4 = tf.Variable(tf.truncated_normal([3, 3, 128, 128], stddev=5e-2))
  b4 = tf.Variable(tf.constant(0.1, shape=[128])) 
  L4 = tf.nn.relu(tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding='SAME') + b4)

  W5 = tf.Variable(tf.truncated_normal([3, 3, 128, 128], stddev=5e-2))
  b5 = tf.Variable(tf.constant(0.1, shape=[128]))
  L5 = tf.nn.relu(tf.nn.conv2d(L4, W5, strides=[1, 1, 1, 1], padding='SAME') + b5)
    
  Fully_W1 = tf.Variable(tf.truncated_normal([8 * 8 * 128, 384], stddev=5e-2))
  Fully_b1 = tf.Variable(tf.constant(0.1, shape=[384]))

  L5 = tf.reshape(L5, [-1, 8*8*128])
  FC = tf.nn.relu(tf.matmul(L5, Fully_W1) + Fully_b1)

  keep_prob = tf.placeholder(tf.float32)
  FC = tf.nn.dropout(FC, keep_prob) 

  Fully_W2 = tf.Variable(tf.truncated_normal([384, 10], stddev=5e-2))
  Fully_b2 = tf.Variable(tf.constant(0.1, shape=[10]))
  FC = tf.matmul(FC,Fully_W2) + Fully_b2

  predict = FC
  return predict, keep_prob


In [3]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y_ = tf.placeholder(tf.float32, [None, 10])
a = 0.001

y_conv, keep_prob = CIFAR(x)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(a).minimize(cross_entropy)

correct_predict = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))

max_steps = 20000 

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  for i in range(max_steps):
    batch = next_batch(128, x_train, y_train_one_hot.eval())

    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={ x: batch[0], y_: batch[1], keep_prob: 1.0})
      loss = cross_entropy.eval(feed_dict={ x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, accuracy %g, loss %g' % (i, train_accuracy, loss))
    
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  test_batch = next_batch(10000, x_test, y_test_one_hot.eval())
  print('test accuracy %g' % accuracy.eval(feed_dict={x: test_batch[0], y_: test_batch[1], keep_prob: 1.0}))

step 0, accuracy 0.078125, loss 273.947
step 100, accuracy 0.351562, loss 1.80744
step 200, accuracy 0.429688, loss 1.75462
step 300, accuracy 0.46875, loss 1.51824
step 400, accuracy 0.476562, loss 1.40224
step 500, accuracy 0.546875, loss 1.35605
step 600, accuracy 0.46875, loss 1.48905
step 700, accuracy 0.5625, loss 1.30924
step 800, accuracy 0.484375, loss 1.35683
step 900, accuracy 0.570312, loss 1.25594
step 1000, accuracy 0.53125, loss 1.2999
step 1100, accuracy 0.570312, loss 1.24158
step 1200, accuracy 0.507812, loss 1.25311
step 1300, accuracy 0.617188, loss 1.05239
step 1400, accuracy 0.625, loss 1.11648
step 1500, accuracy 0.617188, loss 1.09818
step 1600, accuracy 0.609375, loss 1.01981
step 1700, accuracy 0.65625, loss 0.928205
step 1800, accuracy 0.5625, loss 1.15409
step 1900, accuracy 0.648438, loss 1.02674
step 2000, accuracy 0.6875, loss 0.969455
step 2100, accuracy 0.625, loss 1.09033
step 2200, accuracy 0.65625, loss 0.95639
step 2300, accuracy 0.679688, loss 0.81

step 18800, accuracy 0.8125, loss 0.580954
step 18900, accuracy 0.890625, loss 0.34355
step 19000, accuracy 0.835938, loss 0.403102
step 19100, accuracy 0.945312, loss 0.25213
step 19200, accuracy 0.796875, loss 0.557449
step 19300, accuracy 0.875, loss 0.396935
step 19400, accuracy 0.898438, loss 0.344284
step 19500, accuracy 0.867188, loss 0.317854
step 19600, accuracy 0.859375, loss 0.366506
step 19700, accuracy 0.84375, loss 0.492317
step 19800, accuracy 0.851562, loss 0.349676
step 19900, accuracy 0.882812, loss 0.303069
test accuracy 0.6273
